In [3]:
import pandas as pd

In [6]:
df=pd.read_csv('data/gemstone.csv')

In [7]:
df=df.drop(labels=['id'],axis=1)

In [9]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [8]:
## independents and dependent features
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [10]:
X


,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [11]:
X.select_dtypes(exclude='object').columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [18]:
#define which columns should be ordinal-encoded and which should be scaled
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [19]:
#Define the custom ranking for each ordinal
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [21]:
#missing values=nan mean,median,most frequent automated 
from sklearn.impute import SimpleImputer #Handig missing values
from sklearn.preprocessing import StandardScaler # handling freature scaling
from sklearn.preprocessing import OrdinalEncoder #ordinal encoding categorial freature will get ranks use this 
##pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 

In [22]:
##numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
## Categorigal pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)
preprocessor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_cols),
        ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [33]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [34]:
preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  0.87407553,
         1.52872212,  1.35273128],
       [ 0.2351953 ,  1.83363716, -0.12153081, ..., -2.14455824,
        -0.93507064, -0.64678628],
       [ 0.49461699,  0.81585507,  0.39980029, ..., -0.13213573,
         0.29682574,  0.68622543],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ..., -2.14455824,
         0.29682574, -0.64678628],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  0.87407553,
         0.29682574,  0.68622543],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  0.87407553,
         0.29682574, -0.64678628]])

In [35]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())


In [53]:
##model training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [51]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [38]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [39]:
regression.intercept_

array([3970.76628955])

In [50]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [52]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    '''  print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')'''

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- carat
- clarity
- color
- cut
- depth
- ...
Feature names seen at fit time, yet now missing:
- cat_pipeline__clarity
- cat_pipeline__color
- cat_pipeline__cut
- num_pipeline__carat
- num_pipeline__depth
- ...


In [32]:
model_list

[]